In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import time
from urllib.request import urlretrieve
from tqdm import tqdm
from selenium.webdriver import ActionChains
import re

In [13]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

In [2]:
#서울특별시 동대문구 회기동 36-3 경희대학교
#'서울특별시 동대문구 전농동 694 서울시립대학교'
#서울특별시 동대문구 이문동 284-1 한국외국어대학교

In [3]:
def driver_start(university):
    chrome_options = webdriver.ChromeOptions() 
    chrome_options.add_argument('--headless')#창없이 실행
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    path=r'/data/chromedriver'
    driver = webdriver.Chrome(executable_path=path,chrome_options=chrome_options)
    driver.set_window_size(1024,1000)

    url = "https://www.yogiyo.co.kr/mobile/#/"
    driver.get(url)
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="search"]/div/form/input').clear()
    driver.find_element_by_xpath('//*[@id="search"]/div/form/input').send_keys(university)
    driver.find_element_by_xpath('//*[@id="button_search_address"]/button[2]').click()
    time.sleep(1)
    
    return driver

In [4]:
def scroll():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height: #스크롤후 높이 변화 없으면 탈출
            break
        last_height = new_height

In [5]:
def shops_catag(driver,cnt):
    driver.find_element_by_xpath(f'//*[@id="category"]/ul/li[{cnt}]/span').click() #치킨,피자등 카테고리 클릭
    time.sleep(3)

    last_height = driver.execute_script("return document.body.scrollHeight")
    
    scroll()

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    soup2=soup.find_all('div',{'class':"restaurant-list"})[2]
    name_list=[x.text for x in soup2.find_all('div',{'ng-bind':"restaurant.name"})]
    
    return name_list

In [6]:
def scroll_element(shop):#음식점이 있는 곳까지 스크롤
    print(f"{shop} scroll")
    while True:
        last_height = driver.execute_script("return document.body.scrollHeight")
        try:
            some_tag = driver.find_element_by_xpath(f'//*[@title="{shop}"]')
            if some_tag:
                return False
        except Exception:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height: #끝까지 스크롤후 해당 음식점이 없으면 종료 
                return True
            last_height = new_height
            time.sleep(0.5)

In [7]:
def craw(shop,driver):
    try:
        #정보창 클릭
        driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[3]').click()
    except Exception:
        driver.quit()
        driver=driver_start(university)
        time.sleep(3)
        scroll_element(shop)
        #driver.find_element_by_xpath(f'//*[@title="{shop_in[i]}"]').click()
        time.sleep(3)
        driver.find_element_by_xpath(f'//*[@title="{shop}"]').click()
        time.sleep(3)
        driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[3]').click()
    
    time.sleep(2)
    #shop=driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[1]/div[1]/span').text
    star=driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[1]/div[2]/ul/li[1]/span').text
    star=re.sub('[^0-9.]','',star)
    notice=driver.find_element_by_xpath('//*[@id="info"]/div[1]/div[2]').text
    
    return star, notice

In [8]:
university='서울특별시 동대문구 전농동 694 서울시립대학교'
#driver.quit()
driver=driver_start(university)

<ipython-input-3-bb62db29bb57>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=path,chrome_options=chrome_options)


In [9]:
# shops_all=shops_catag(driver,2)
# shops_conv=shops_catag(driver,14)
# shops_list=list(set(shops_all)-set(shops_conv))
# len(shops_list)

In [10]:
# shop_list=pd.Series(shops_list)
# path='/data/yogiyo/pr_data/uos/'
# shop_list.to_pickle(path+'only_shop_name.pickle')

In [11]:
path='/data/yogiyo/pr_data/uos/'
shops_list=pd.read_pickle(path+'only_shop_name.pickle').tolist()
len(shops_list)

2057

In [14]:
start=645
end=700
#end=len(shops_list)

# shops=[]
# notices=[]
# stars=[]
# review_cnts=[]
driver.quit()
driver=driver_start(university)
time.sleep(2)
#전체보기로 이동
driver.find_element_by_xpath(f'//*[@id="category"]/ul/li[2]/span').click()
time.sleep(2)
for i in tqdm(range(start,end)):
    shop=shops_list[i]
    print(f'shop{i} : {shop}')
    #음식점 있는 곳까지 스크롤
    go_stop=scroll_element(shop)
    if go_stop:
        #스크롤 했는데 음식점 없으면 다음으로 넘어감
        print('wrong shop')
        continue
        
    time.sleep(3)
    
    #찾은 음식점 클릭
    try:
        driver.find_element_by_xpath(f'//*[@title="{shop}"]').click()
    except (Exception, NoSuchElementException, ElementClickInterceptedException):
        driver.quit()
        driver=driver_start(university)
        time.sleep(2)
        scroll_element(shop)
        time.sleep(3)
        driver.find_element_by_xpath(f'//*[@title="{shop}"]').click()
    time.sleep(3)
    
    #식당의 리뷰가 없으면 넘어감
    review_cnt=driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a/span').text
    if review_cnt=='0':
        print('리뷰가 없습니다')
        driver.back()
        time.sleep(2)
        continue
    
    star,notice=craw(shop,driver)
    shops.append(shop)
    stars.append(star)
    review_cnts.append(review_cnt)
    print(shop,star,review_cnt)
    notices.append(notice)
    driver.back()
    time.sleep(2)
    
df=pd.DataFrame({'shop':shops,'star':stars,'review_cnt':review_cnts,'notice':notices})

<ipython-input-3-bb62db29bb57>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=path,chrome_options=chrome_options)
  0%|          | 0/55 [00:00<?, ?it/s]

shop645 : 퍼스트에스프레소
퍼스트에스프레소 scroll
리뷰가 없습니다


  2%|▏         | 1/55 [00:32<28:51, 32.06s/it]

shop646 : 커피마마장안우리집
커피마마장안우리집 scroll
리뷰가 없습니다


  4%|▎         | 2/55 [00:57<26:36, 30.12s/it]

...

 96%|█████████▋| 53/55 [24:41<00:54, 27.03s/it]

wrong shop
shop698 : 파파존스피자-장안점
파파존스피자-장안점 scroll
파파존스피자-장안점 scroll
파파존스피자-장안점 4.8 73


 98%|█████████▊| 54/55 [25:21<00:30, 30.94s/it]

shop699 : 큐데이블디저트카페
큐데이블디저트카페 scroll
큐데이블디저트카페 5.0 128


100%|██████████| 55/55 [25:46<00:00, 28.12s/it]


In [15]:
df.to_csv(path+'uos_shop1.csv')